# Extract and Format Data from Downloaded Files

Take the data downloaded from Yahoo and transform into a single dictionary keyed on ticker symbol. 

Assign sector and industry for each company.

Save data as pickle files.

In [1]:
import pandas as pd
import numpy as np
from os import listdir
import re

import csv
import datetime
import pickle

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
def import_data():

  data_root = "/mnt/data/projects/TS1/data/R1000/Yahoo"

  master_company_data = {}
  master_company_profile = {}

  statement_type = {'balance': 'balanceSheetHistory',
                    'income': 'incomeStatementHistory',
                    'cash': 'cashflowStatementHistory'}
  
  # Loop through each statement type (balance,income,cash)
                    
  for stype, sdesc in statement_type.items():
    
    # Loop through each of the files. we have 4 numbered 1 to 4
                    
    for i in range(1,5): 
      
      # Read data from pickle file. Data is in the form of a dictionary.
                    
      filename=data_root+'/Yahoo_'+stype+'_statements_g'+str(i)+'.pkl'
      statement_dict = pd.read_pickle(filename)
    
      # Data downloaded from Yahoo is organised into sectors and then ticker. 
      # Hence loop through each sector and ticker.
    
      for sector in statement_dict:
        print(sector)
        
        for ticker in statement_dict[sector][sdesc]:
        
          print(ticker)
          statement=pd.DataFrame() 
        
          # Can't figure out a more elegant way to do this. We want to set entry
          # for ticker to dictionary on first iteration so that that we can store 
          # the three statements (balance, income, cash)
            
          if stype == 'balance':
            master_company_data[ticker] = {}
                    
          no_entries=len(statement_dict[sector][sdesc][ticker])
        
          # Build statement table 
                    
          company_data = {}
        
          for i in range(0,no_entries):    
            df = pd.DataFrame.from_dict(statement_dict[sector][sdesc][ticker][i])
            statement=pd.concat([statement,df],axis=1)
        
          # Store statement for company
            
          master_company_data[ticker][stype] = statement
  
  master_company_data.pop('CAT')
  
  return master_company_data

In [4]:
def import_company_profile_data():
    
  data_root = "/mnt/data/projects/TS1/data/R1000/Yahoo"

  master_company_profile = {}

  for i in range(1,5):

    filename=data_root+'/Yahoo_company_profile_g'+str(i)+'.pkl'
    company_profile = pd.read_pickle(filename)

    for sector in company_profile:
      for ticker in company_profile[sector]:
        print(ticker)
        
        # We failed to download profile data for CAT so ignore
        
        if ticker != 'CAT':
          master_company_profile[ticker] = company_profile[sector][ticker]
          master_company_profile[ticker]['gics_sector'] = sector
 
  return master_company_profile


In [5]:
master_company_data = import_data()

Energy
AM
AR
APA
BKR
LNG
CHK
CVX
COP
CTRA
DVN
FANG
DTM
EVA
EOG
EQT
XOM
HAL
HES
DINO
KMI
MRO
MPC
NFE
NOV
OXY
OKE
OVV
PDCE
PSX
PXD
RRC
SLB
SWN
TRGP
TPL
VLO
VTS
WMB
Utilities
AES
LNT
AEE
AEP
AWK
ATO
AGR
BEPC
CNP
CMS
ED
CEG
D
DTE
DUK
EIX
ETR
WTRG
EVRG
ES
EXC
FE
HE
IDA
MDU
NFG
NEE
NI
NRG
OGE
PCG
PNW
PPL
PEG
SRE
SO
UGI
VST
WEC
XEL
Communication Services
ATVI
GOOG
ATUS
AMC
T
CABO
CHTR
CMCSA
DISH
DIS
EA
FOXA
FOX
FYBR
IAC
IPG
LBRDA
LBRDK
FWONA
FWONK
LSXMA
LSXMK
LYV
LUMN
MSGS
MTCH
META
NFLX
NYT
NWSA
NWS
NXST
OMC
PARAA
PARA
PINS
PLTK
RBLX
ROKU
SIRI
SPOT
TMUS
TTWO
TRIP
VZ
WBD
WWE
ZI
Consumer Staples
ADM
ACI
MO
BJ
SAM
BF-A
BG
CPB
CASY
CHD
CLX
KO
CL
CAG
STZ
COST
COTY
DAR
DG
DLTR
EL
FLO
FRPT
GIS
GO
HSY
HRL
INGR
K
KDP
KMB
KHC
KR
LW
MKC
TAP
MDLZ
MNST
OLPX
PEP
PFGC
PM
PPC
POST
PG
REYN
SEB
SJM
SPB
SYY
TGT
TSN
USFD
WBA
WMT
Materials
APD
ALB
AA
AMCR
ATR
AMBP
ASH
AVY
AXTA
BALL
BERY
CE
CF
CC
CLF
CTVA
CCK
DOW
DD
EXP
EMN
ECL
ESI
FMC
FCX
DNA
GPK
HUN
IFF
IP
LIN
LPX
LYB
MLM
MOS
MP
NEU
NEM
NUE
OLN
PKG
PPG
RS
RGLD


DG
DLTR
EL
FLO
FRPT
GIS
GO
HSY
HRL
INGR
K
KDP
KMB
KHC
KR
LW
MKC
TAP
MDLZ
MNST
OLPX
PEP
PFGC
PM
PPC
POST
PG
REYN
SEB
SJM
SPB
SYY
TGT
TSN
USFD
WBA
WMT
Materials
APD
ALB
AA
AMCR
ATR
AMBP
ASH
AVY
AXTA
BALL
BERY
CE
CF
CC
CLF
CTVA
CCK
DOW
DD
EXP
EMN
ECL
ESI
FMC
FCX
DNA
GPK
HUN
IFF
IP
LIN
LPX
LYB
MLM
MOS
MP
NEU
NEM
NUE
OLN
PKG
PPG
RS
RGLD
RPM
SMG
SEE
SHW
SLGN
SON
SCCO
SSRM
STLD
X
VVV
VMC
WLK
WRK
Real Estate
ARE
AMH
AMT
COLD
AIRC
AVB
BXP
BRX
CPT
CBRE
CUZ
CCI
CUBE
DLR
DEI
EGP
EPR
EQIX
ELS
EQR
ESS
EXR
FRT
FR
GLPI
HR
PEAK
HIW
HST
HHC
INVH
IRM
JBGS
JLL
KRC
KIM
LAMR
LSI
MPW
MAA
NNN
NSA
OHI
OPEN
PK
PLD
PSA
RYN
O
REG
REXR
SBAC
SPG
SLG
SRC
SUI
UDR
VTR
VICI
VNO
WPC
WELL
WE
WY
ZG
Z
Industrials
MMM
AYI
ADP
WMS
ACM
AGCO
AL
ALK
ALLE
ALSN
AAL
AME
AWI
CAR
AXON
AZEK
BA
BAH
BR
BLDR
BWXT
CHRW
CACI
CSL
CARR
CAT
CHPT
CTAS
CLVT
CLH
CPA
CPRT
CNM
CSGP
CR
CXT
CSX
CMI
CW
DE
DAL
DCI
DOV
DRVN
DNB
ETN
EMR
EFX
ESAB
EXPD
FAST
FDX
FLS
FTV
FBIN
FCN
GTES
GE
GNRC
GD
GGG
GWW
GXO
HAYW
HEI
HTZ
HXL
HON
HWM
HUBB
HII
IEX
ITW
IR
ITT


In [6]:
master_company_profile = import_company_profile_data()

AM
AR
APA
BKR
LNG
CHK
CVX
COP
CTRA
DVN
FANG
DTM
EVA
EOG
EQT
XOM
HAL
HES
DINO
KMI
MRO
MPC
NFE
NOV
OXY
OKE
OVV
PDCE
PSX
PXD
RRC
SLB
SWN
TRGP
TPL
VLO
VTS
WMB
AES
LNT
AEE
AEP
AWK
ATO
AGR
BEPC
CNP
CMS
ED
CEG
D
DTE
DUK
EIX
ETR
WTRG
EVRG
ES
EXC
FE
HE
IDA
MDU
NFG
NEE
NI
NRG
OGE
PCG
PNW
PPL
PEG
SRE
SO
UGI
VST
WEC
XEL
ATVI
GOOG
ATUS
AMC
T
CABO
CHTR
CMCSA
DISH
DIS
EA
FOXA
FOX
FYBR
IAC
IPG
LBRDA
LBRDK
FWONA
FWONK
LSXMA
LSXMK
LYV
LUMN
MSGS
MTCH
META
NFLX
NYT
NWSA
NWS
NXST
OMC
PARAA
PARA
PINS
PLTK
RBLX
ROKU
SIRI
SPOT
TMUS
TTWO
TRIP
VZ
WBD
WWE
ZI
ADM
ACI
MO
BJ
SAM
BF-A
BG
CPB
CASY
CHD
CLX
KO
CL
CAG
STZ
COST
COTY
DAR
DG
DLTR
EL
FLO
FRPT
GIS
GO
HSY
HRL
INGR
K
KDP
KMB
KHC
KR
LW
MKC
TAP
MDLZ
MNST
OLPX
PEP
PFGC
PM
PPC
POST
PG
REYN
SEB
SJM
SPB
SYY
TGT
TSN
USFD
WBA
WMT
APD
ALB
AA
AMCR
ATR
AMBP
ASH
AVY
AXTA
BALL
BERY
CE
CF
CC
CLF
CTVA
CCK
DOW
DD
EXP
EMN
ECL
ESI
FMC
FCX
DNA
GPK
HUN
IFF
IP
LIN
LPX
LYB
MLM
MOS
MP
NEU
NEM
NUE
OLN
PKG
PPG
RS
RGLD
RPM
SMG
SEE
SHW
SLGN
SON
SCCO
SSRM
STLD
X
VVV
VMC
WLK
WRK
ARE
AMH
A

# Assign sector and industry

In [7]:
for ticker in master_company_profile:
  print(ticker)
  master_company_data[ticker]['industry'] = master_company_profile[ticker]['industry']
  master_company_data[ticker]['yahoo_sector'] = master_company_profile[ticker]['sector']
  master_company_data[ticker]['gics_sector'] = master_company_profile[ticker]['gics_sector']

AM
AR
APA
BKR
LNG
CHK
CVX
COP
CTRA
DVN
FANG
DTM
EVA
EOG
EQT
XOM
HAL
HES
DINO
KMI
MRO
MPC
NFE
NOV
OXY
OKE
OVV
PDCE
PSX
PXD
RRC
SLB
SWN
TRGP
TPL
VLO
VTS
WMB
AES
LNT
AEE
AEP
AWK
ATO
AGR
BEPC
CNP
CMS
ED
CEG
D
DTE
DUK
EIX
ETR
WTRG
EVRG
ES
EXC
FE
HE
IDA
MDU
NFG
NEE
NI
NRG
OGE
PCG
PNW
PPL
PEG
SRE
SO
UGI
VST
WEC
XEL
ATVI
GOOG
ATUS
AMC
T
CABO
CHTR
CMCSA
DISH
DIS
EA
FOXA
FOX
FYBR
IAC
IPG
LBRDA
LBRDK
FWONA
FWONK
LSXMA
LSXMK
LYV
LUMN
MSGS
MTCH
META
NFLX
NYT
NWSA
NWS
NXST
OMC
PARAA
PARA
PINS
PLTK
RBLX
ROKU
SIRI
SPOT
TMUS
TTWO
TRIP
VZ
WBD
WWE
ZI
ADM
ACI
MO
BJ
SAM
BF-A
BG
CPB
CASY
CHD
CLX
KO
CL
CAG
STZ
COST
COTY
DAR
DG
DLTR
EL
FLO
FRPT
GIS
GO
HSY
HRL
INGR
K
KDP
KMB
KHC
KR
LW
MKC
TAP
MDLZ
MNST
OLPX
PEP
PFGC
PM
PPC
POST
PG
REYN
SEB
SJM
SPB
SYY
TGT
TSN
USFD
WBA
WMT
APD
ALB
AA
AMCR
ATR
AMBP
ASH
AVY
AXTA
BALL
BERY
CE
CF
CC
CLF
CTVA
CCK
DOW
DD
EXP
EMN
ECL
ESI
FMC
FCX
DNA
GPK
HUN
IFF
IP
LIN
LPX
LYB
MLM
MOS
MP
NEU
NEM
NUE
OLN
PKG
PPG
RS
RGLD
RPM
SMG
SEE
SHW
SLGN
SON
SCCO
SSRM
STLD
X
VVV
VMC
WLK
WRK
ARE
AMH
A

## Save Data to Pickle File

In [8]:
with open('/home/priyesh/projects/TS1/pickle/yahoo_company_data.pkl','wb') as f:
    pickle.dump(master_company_data,f)
    
with open('/home/priyesh/projects/TS1/pickle/yahoo_company_profile.pkl','wb') as f:
    pickle.dump(master_company_profile,f)

In [12]:
len(master_company_profile.keys())

848